# CRIM Intervals:  Modules

### What You Can Do with this Notebook

* Learn how to do some basic filtering of one Dataframe to find the modules associated with homorhythmic passages



### A. Import Intervals and Other Code


In [35]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os


def convertTuple(tup):
    out = ""
    if isinstance(tup, tuple):
        out = ', '.join(tup)
    return out


MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [92]:
a = ['Josquin', "Fevin", "Josquin"]
set(a)
a[0]

'Josquin'

## B. Importing a Piece

In [70]:
# Select a prefix:

# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/'

# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# and combine the strings and load the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

Previously imported piece detected.
{'title': 'Ave Maria', 'composer': 'Josquin Des Prés'}


In [39]:
# print(ImportedPiece.ngrams.__doc__)

### How can we find the ngrams associated with all the HR in a piece?

* We will first find the homorhythmic passages (where two or more voices have the same durations and same syllables).

* Then we will use the 'offsets' of those passages as a mask, returning only the harmonic ngrams associated with them.

* And then we can look for repeating harmonic ngrams in one or more pieces.



### Find HR, then Indices to List

find hr and then get the offset index as a list
hr has a 'multi'index, so we only need level 2, which is the offset

>`hr = piece.homorhythm()
hr_index = hr.index.get_level_values(2)`


In [71]:
notes = piece.notes()
notes.index.to_list()

[0.0,
 4.0,
 12.0,
 16.0,
 20.0,
 24.0,
 28.0,
 32.0,
 36.0,
 40.0,
 44.0,
 48.0,
 52.0,
 56.0,
 60.0,
 62.0,
 64.0,
 68.0,
 72.0,
 74.0,
 76.0,
 78.0,
 80.0,
 82.0,
 83.0,
 84.0,
 88.0,
 91.0,
 92.0,
 94.0,
 96.0,
 98.0,
 99.0,
 100.0,
 104.0,
 106.0,
 108.0,
 110.0,
 112.0,
 114.0,
 115.0,
 116.0,
 120.0,
 124.0,
 126.0,
 128.0,
 130.0,
 132.0,
 136.0,
 140.0,
 144.0,
 146.0,
 147.0,
 148.0,
 150.0,
 152.0,
 156.0,
 158.0,
 160.0,
 161.0,
 162.0,
 164.0,
 166.0,
 168.0,
 172.0,
 176.0,
 178.0,
 180.0,
 181.0,
 182.0,
 184.0,
 186.0,
 188.0,
 192.0,
 195.0,
 196.0,
 200.0,
 202.0,
 204.0,
 208.0,
 210.0,
 211.0,
 212.0,
 215.0,
 216.0,
 217.0,
 218.0,
 219.0,
 220.0,
 221.0,
 222.0,
 224.0,
 225.0,
 226.0,
 228.0,
 230.0,
 232.0,
 238.0,
 240.0,
 244.0,
 248.0,
 252.0,
 256.0,
 260.0,
 262.0,
 265.0,
 266.0,
 268.0,
 270.0,
 272.0,
 276.0,
 280.0,
 284.0,
 288.0,
 292.0,
 294.0,
 297.0,
 298.0,
 300.0,
 302.0,
 304.0,
 312.0,
 316.0,
 320.0,
 324.0,
 328.0,
 331.0,
 332.0,
 334.0,
 33

In [83]:
hr = piece.homorhythm()
hr_index = hr.index.get_level_values(2)

hr_index
hr.head(50)


,,,active_voices,number_dur_ngrams,[Superius],Altus,Tenor,Bassus,syllable_set,count_lyr_ngrams,active_syll_voices,voice_match
Measure,Beat,Offset,,,,,,,,,,
31,3.0,244.0,2.0,1.0,"(A, ve)","(A, ve)",NaN,NaN,"[(A, ve), (A, ve)]",1.0,2.0,True
32,3.0,252.0,2.0,1.0,"(cae, lo)","(cae, lo)",NaN,NaN,"[(cae, lo), (cae, lo)]",1.0,2.0,True
33,1.0,256.0,2.0,1.0,"(lo, rum)","(lo, rum)",NaN,NaN,"[(lo, rum), (lo, rum)]",1.0,2.0,True
35,3.0,276.0,2.0,1.0,NaN,NaN,"(A, ve)","(A, ve)","[(A, ve), (A, ve)]",1.0,2.0,True
38,4.0,302.0,2.0,1.0,NaN,"(mi, na)","(mi, na)",NaN,"[(mi, na), (mi, na)]",1.0,2.0,True
39,1.0,304.0,3.0,1.0,NaN,"(na, Ma)","(na, Ma)","(na, Ma)","[(na, Ma), (na, Ma), (na, Ma)]",1.0,3.0,True
40,3.0,316.0,4.0,1.0,"(Ma, ri)","(Ma, ri)","(Ma, ri)","(Ma, ri)","[(Ma, ri), (Ma, ri), (Ma, ri), (Ma, ri)]",1.0,4.0,True
41,1.0,320.0,4.0,1.0,"(ri, a)","(ri, a)","(ri, a)","(ri, a)","[(ri, a), (ri, a), (ri, a), (ri, a)]",1.0,4.0,True
52,1.0,408.0,3.0,1.0,NaN,"(ti, ti)",NaN,"(ti, ti)","[(ti, ti), (ti, ti)]",1.0,2.0,False


### Find all the harmonic ngrams in one piece (note interval and offset settings, and length)

>`all_har_ngs = piece.ngrams(df=piece.harmonic(kind="d", compound=False), n=2).fillna('')`

if you want the **modular ngrams** instead, us the following:

>`all_mods = piece.ngrams()`

In [73]:
dur = piece.lyrics()
piece.ngrams(df=dur, n=2)

,[Superius],Altus,Tenor,Bassus
0.0,"(A-, -ve)",NaN,NaN,NaN
4.0,"(-ve, Ma-)",NaN,NaN,NaN
12.0,"(Ma-, -ri-)",NaN,NaN,NaN
16.0,"(-ri-, -a,)","(A-, -ve)",NaN,NaN
20.0,NaN,"(-ve, Ma-)",NaN,NaN
...,...,...,...,...
1240.0,"(-men-, -to)","(-men-, -to)","(-men-, -to)","(-men-, -to)"
1244.0,"(-to, me-)","(-to, me-)","(-to, me-)","(-to, me-)"
1248.0,"(me-, -i.)","(me-, -i.)","(me-, -i.)","(me-, -i.)"
1256.0,"(-i., A-)","(-i., A-)","(-i., A-)","(-i., A-)"


In [74]:
piece.ngrams()

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,NaN,NaN,NaN,NaN,NaN,"12_4, 10_Held, 8"
20.0,NaN,NaN,NaN,NaN,NaN,"10_Held, 8_1, 8"
32.0,NaN,NaN,NaN,"5_4, 3_Held, 1",NaN,NaN
36.0,NaN,NaN,NaN,"3_Held, 1_1, 1",NaN,NaN
48.0,"12_4, 10_Held, 8",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1240.0,"1_-4, 3_2, 3","5_-4, 8_2, 5","10_-4, 12_2, 10","5_-2, 6_2, 3","10_-2, 10_2, 8","6_1, 5_-3, 6"
1244.0,"3_2, 3_-2, 5","8_2, 5_Held, 6","12_2, 10_-2, 10","6_2, 3_Held, 4","10_2, 8_2, 6","5_-3, 6_2, 5"
1248.0,"3_-2, 5_-5, 8","5_Held, 6_-2, 8","10_-2, 10_-5, 15","3_Held, 4_2, 4","8_2, 6_-2, 8","6_2, 5_2, 3"
1252.0,NaN,"6_-2, 8_-5, 12",NaN,"4_2, 4_-2, 5",NaN,"5_2, 3_1, 4"


In [75]:
all_har_ngs = piece.ngrams(df=piece.harmonic(kind="d", compound=False), n=2).fillna('')
all_har_ngs

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"(5, 3)"
20.0,,,,,,"(3, 8)"
24.0,,,,,,"(8, 8)"
32.0,,,,"(5, 3)",,
36.0,,,,"(3, 1)",,
...,...,...,...,...,...,...
1244.0,"(3, 3)","(8, 5)","(5, 3)","(6, 3)","(3, 8)","(5, 6)"
1248.0,"(3, 5)","(5, 6)","(3, 3)","(3, 4)","(8, 6)","(6, 5)"
1252.0,,"(6, 8)",,"(4, 4)",,"(5, 3)"
1256.0,"(5, 8)","(8, 5)","(3, 8)","(4, 5)","(6, 8)","(3, 4)"


#### Now get a list of modules

>`all_ngs_list = all_har_ngs.index.to_list()`

And find the **intersection of both lists** (since sometimes the hr don't form modules, depending on surrounding rests, etc)

>`final_hr_list= hr_index.intersection(all_ngs_list)`



In [76]:
all_ngs_list = all_har_ngs.index.to_list()
final_hr_list= hr_index.intersection(all_ngs_list)
len(final_hr_list)


29

### Now filter the Ngrams according to the index of HR passages

>`all_hr_ngrams = all_har_ngs.loc[final_hr_list]
all_hr_ngrams.fillna('')
all_hr_ngrams = all_hr_ngrams.fillna('').applymap(convertTuple)`

These are the ngrams of the hr passages

In [77]:

all_hr_ngrams = all_har_ngs.loc[final_hr_list]
all_hr_ngrams = all_hr_ngrams.fillna('').applymap(convertTuple)
all_hr_ngrams

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
Offset,,,,,,
244.0,,,,,,"8, 6"
252.0,,,,,,"6, 6"
256.0,,,,,,"6, 6"
276.0,"8, 6",,,,,
302.0,"6, 8","3, 5",,"-4, -4",,
316.0,"5, 5","8, 8","3, 3","-5, -5","6, 6","3, 3"
320.0,"5, 5","8, 8","3, 3","-5, -5","6, 6","3, 3"
408.0,"5, 8","3, 3","5, 4","-3, -6","8, 7","3, 2"
604.0,"5, 8",,"4, 3",,"7, 6",


### There are various ways to change the of NGrams 

You would need to use this with the hr mask above

- Diatonic, Chromatic, With-Quality, etc
- No Unisons
- Various lengths
- Exclude rests

In [78]:
mel = piece.melodic(kind="c")
har = piece.harmonic(kind="c")
ngrams = piece.ngrams(df=har, other=mel, n=3).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"19_5, 16_Held, 12"
20.0,,,,,,"16_Held, 12_0, 12"
32.0,,,,"7_5, 4_Held, 0",,
36.0,,,,"4_Held, 0_0, 0",,
48.0,"19_5, 16_Held, 12",,,,,


In [79]:
nr_no_unisons = piece.notes(combineUnisons=True)
mel = piece.melodic(df=nr_no_unisons, kind="d")
har = piece.harmonic(kind="d", compound=False)
ngrams = piece.ngrams(df=har, other=mel).fillna('')
ngrams.head()

,Bassus_Tenor,Bassus_Altus,Bassus_[Superius],Tenor_Altus,Tenor_[Superius],Altus_[Superius]
16.0,,,,,,"5_4, 3_Held, 8"
20.0,,,,,,"3_Held, 8_Held, 8"
32.0,,,,"5_4, 3_Held, 1",,
36.0,,,,"3_Held, 1_Held, 1",,
48.0,"5_4, 3_Held, 8",,,,,


## Search the Ngrams of the HR 

Now you can also search within those results for anything you like

In [80]:

@interact
def har_ngram_search(my_search="", df = fixed(all_hr_ngrams)):
    
    df2 = all_hr_ngrams.copy()
    filtered_ngrams = df2[df2.apply(lambda x: x.astype(str).str.contains(my_search).any(), axis=1)].copy()
    
    pd.set_option('max_columns', None)
    return filtered_ngrams.fillna("-").reset_index().applymap(str).style.applymap(lambda x: "background: #ccebc4" if re.search(my_search, x) else "")

interactive(children=(Text(value='', description='my_search'), Output()), _dom_classes=('widget-interact',))

In [81]:
piece.verovioHomorhythm()

Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  31
HR Stop Measure:  33
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  2


[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported data.PERCENT '100'
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] Unsupported '<line>' within <measure>
[Warning] 4 time pointing element(s) could not be matched in measure m-4149
[Warning] 4 time pointing element(s) could not be matched in measure m-4132
[Warning] 4 time pointing element(s) could not be matched in measure m-4098


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  35
HR Stop Measure:  35
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  2


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  38
HR Stop Measure:  41
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  4


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  52
HR Stop Measure:  52
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  2


...

Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  123
HR Stop Measure:  123
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  2


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  127
HR Stop Measure:  127
Minimum Number of HR Voices:  2
Maximum Number of HR Voices:  2


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  141
HR Stop Measure:  141
Minimum Number of HR Voices:  3
Maximum Number of HR Voices:  3


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  143
HR Stop Measure:  144
Minimum Number of HR Voices:  4
Maximum Number of HR Voices:  4


Results:
File Name:  CRIM_Model_0008
Josquin Des Prés
Ave Maria
HR Start Measure:  146
HR Stop Measure:  148
Minimum Number of HR Voices:  4
Maximum Number of HR Voices:  4
